In [1]:
import os
import sys
import csv
import pandas as pd
import numpy as np

import os.path
from os import path

import PIL
from PIL import Image

import cv2
import glob
import h5py
import os
import pathlib
import tables

import torch
from skimage import io, transform
from sklearn import metrics
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from matplotlib import pyplot

import torch
import timm
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time
import torch.nn.functional as F
import torch.nn as nn
from torch.nn.functional import kl_div, softmax, log_softmax
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point, Polygon
import geoplot


/home/irene/myenv/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = '/home/irene/ISEF-Project-2023/densityresnet50best_model.pth'

In [3]:
hdf5_path = '/home/irene/ISEF-Project-2023/Testing/NJ/DensityNJ.hdf5'
file = h5py.File(hdf5_path, "r")
file.keys()

<KeysViewHDF5 ['test_img', 'test_path']>

In [4]:
rgb_mean,rgb_std = (0.485, 0.456, 0.406),(0.229, 0.224, 0.225)

In [5]:
class HDF5Dataset(Dataset):


    def __init__(self, h5_path, set_name, transform = None):
        self.h5_path = h5_path
        self.dataset = None
        self.transform = transform
        self.file_path  = h5_path
        self.set = set_name
        
        str_name = self.set + "_img"
        
        file = h5py.File(h5_path, "r")
        self.dataset_len = len(file[str_name])
        
        self.transform = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.Resize(224),
                transforms.ToTensor(),
                transforms.Normalize(rgb_mean,rgb_std)
            ])
        
    def __getitem__(self, index): #to enable indexing
        
        if self.dataset is None:
            self.imgs = h5py.File(self.file_path, 'r')[self.set + "_img"]
           
            path= self.file_path
            cur_img = self.imgs[index]
            PIL_image = Image.fromarray(np.uint8(cur_img)).convert('RGB')#3 channels don't need alpha channel network input
            image = self.transform(PIL_image)

            
            
        return (image, path)

    def __len__(self):
        return self.dataset_len

In [6]:
from torch.utils.data import Dataset,DataLoader

test_loader = DataLoader(HDF5Dataset(h5_path=hdf5_path,set_name="test"), batch_size=8,shuffle=True,drop_last=False)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

out_dim = 2


In [8]:
def get_inference (net, test_loader, a_device = None):
    
    iterator = iter(test_loader)
    total_step = len(test_loader)
    
    print(total_step)
    net.eval()
    
    with torch.no_grad():
        record = 0
        probs_all = []
        predictions_all = []
        for step in range(total_step-1):
            images,paths = next(iterator)
            images.to(a_device)
            record += images.shape[0]
            images = images.to(a_device)
            
            output = net(images)
            probs = torch.nn.functional.softmax(output, dim=1)

            _, preds = torch.max(output.data,1)
            
            if(len(predictions_all) ==0):
                predictions_all = preds.detach().cpu().numpy()
                probs_all = probs.detach().cpu().numpy()
            else:
                predictions_all = np.vstack((predictions_all, preds.detach().cpu().numpy())) 
                probs_all = np.vstack((probs_all, probs.detach().cpu().numpy()))
            print(record)
    
    return probs_all, predictions_all #add this later
                
        #y_test --> label, y_score --> probs all 

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



model = timm.create_model("resnet50", pretrained=True)


model.to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.001,weight_decay = 1e-4,momentum = 0.9)
criterion = nn.CrossEntropyLoss()

In [10]:
my_dict = torch.load(model_path)

In [11]:
model.load_state_dict(my_dict)

<All keys matched successfully>

In [12]:
y_prob, y_pred= get_inference (model, test_loader, a_device = device)


2236
8
16
24
32
40
48
56
64
72
80
88
96
104
112
120
128
136
144
152
160
168
176
184
192
200
208
216
224
232
240
248
256
264
272
280
288
296
304
312
320
328
336
344
352
360
368
376
384
392
400
408
416
424
432
440
448
456
464
472
480
488
496
504
512
520
528
536
544
552
560
568
576
584
592
600
608
616
624
632
640
648
656
664
672
680
688
696
704
712
720
728
736
744
752
760
768
776
784
792
800
808
816
824
832
840
848
856
864
872
880
888
896
904
912
920
928
936
944
952
960
968
976
984
992
1000
1008
1016
1024
1032
1040
1048
1056
1064
1072
1080
1088
1096
1104
1112
1120
1128
1136
1144
1152
1160
1168
1176
1184
1192
1200
1208
1216
1224
1232
1240
1248
1256
1264
1272
1280
1288
1296
1304
1312
1320
1328
1336
1344
1352
1360
1368
1376
1384
1392
1400
1408
1416
1424
1432
1440
1448
1456
1464
1472
1480
1488
1496
1504
1512
1520
1528
1536
1544
1552
1560
1568
1576
1584
1592
1600
1608
1616
1624
1632
1640
1648
1656
1664
1672
1680
1688
1696
1704
1712
1720
1728
1736
1744
1752
1760
1768
1776
1784
1792
1800
1808
18

12816
12824
12832
12840
12848
12856
12864
12872
12880
12888
12896
12904
12912
12920
12928
12936
12944
12952
12960
12968
12976
12984
12992
13000
13008
13016
13024
13032
13040
13048
13056
13064
13072
13080
13088
13096
13104
13112
13120
13128
13136
13144
13152
13160
13168
13176
13184
13192
13200
13208
13216
13224
13232
13240
13248
13256
13264
13272
13280
13288
13296
13304
13312
13320
13328
13336
13344
13352
13360
13368
13376
13384
13392
13400
13408
13416
13424
13432
13440
13448
13456
13464
13472
13480
13488
13496
13504
13512
13520
13528
13536
13544
13552
13560
13568
13576
13584
13592
13600
13608
13616
13624
13632
13640
13648
13656
13664
13672
13680
13688
13696
13704
13712
13720
13728
13736
13744
13752
13760
13768
13776
13784
13792
13800
13808
13816
13824
13832
13840
13848
13856
13864
13872
13880
13888
13896
13904
13912
13920
13928
13936
13944
13952
13960
13968
13976
13984
13992
14000
14008
14016
14024
14032
14040
14048
14056
14064
14072
14080
14088
14096
14104
14112
14120
14128
14136
1414

In [13]:
nj = pd.read_csv("DensityNJ.csv")

In [14]:
nj["pred"] = ""

In [15]:
y_pred

array([[  0,   0,   0, ...,   0,   1,   0],
       [  1,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   1,   0,   0],
       ...,
       [  0,   1,   0, ...,   0,   0,   0],
       [610,   1,   0, ...,   0,   0, 610],
       [  0,   0, 610, ...,   0, 610,   0]])

In [16]:
ind = 0
for i in range(len(y_pred)):
    for j in range(8):
        nj.at[ind, 'pred'] = y_pred[i][j]
        ind += 1

In [17]:
nj

,Unnamed: 0,Path,Upper,Lower,pred
0,0,/home/irene/ISEF-Project-2023/Data/DensityNJ/1...,"40.1918077468872, -75.1309382915497","40.180059671402, -75.1196622848511",0
1,1,/home/irene/ISEF-Project-2023/Data/DensityNJ/1...,"41.2373864650726, -75.1309382915497","41.2256383895874, -75.1196622848511",0
2,2,/home/irene/ISEF-Project-2023/Data/DensityNJ/1...,"41.2256383895874, -75.1309382915497","41.2138903141022, -75.1196622848511",0
3,3,/home/irene/ISEF-Project-2023/Data/DensityNJ/1...,"41.2138903141022, -75.1309382915497","41.2021422386169, -75.1196622848511",0
4,4,/home/irene/ISEF-Project-2023/Data/DensityNJ/1...,"41.2021422386169, -75.1309382915497","41.1903941631317, -75.1196622848511",0
...,...,...,...,...,...
17877,17877,/home/irene/ISEF-Project-2023/Data/DensityNJ/2...,"40.1406741142273, -74.2863428592682","40.1288402080536, -74.2744874954224",0
17878,17878,/home/irene/ISEF-Project-2023/Data/DensityNJ/2...,"40.1288402080536, -74.2863428592682","40.1170063018799, -74.2744874954224",610
17879,17879,/home/irene/ISEF-Project-2023/Data/DensityNJ/2...,"40.1170063018799, -74.2863428592682","40.1051723957062, -74.2744874954224",0
17880,17880,/home/irene/ISEF-Project-2023/Data/DensityNJ/2...,"40.1051723957062, -74.2863428592682","40.0933384895325, -74.2744874954224",


In [18]:
nj.to_csv('DensityGDF.csv')

In [ ]:
nj